In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math
import optuna
# 加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

In [9]:
class Custom(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = nn.Parameter(torch.tensor(sigma))  # 1个可学习参数 sigma
    def forward(self, input):
        temp = input * (1 + torch.erf(input / self.sigma)) / 2  # 前向传播
        return temp

class NetGELU(nn.Module):
    def __init__(self,sigma):
        super(NetGELU, self).__init__()
        self.flatten = nn.Flatten()  # 28*28二维图像展平成一维向量，size=784
        self.fc1 = nn.Linear(28*28, 30)  # 784维映射到50维
        self.custom1 = Custom(sigma)  # 自定义激活函数层
        self.dropout = nn.Dropout(0.5)  # 舍去50%的神经元
        self.fc2 = nn.Linear(30, 10)  # 50维映射到10维

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.custom1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [14]:
def objective(trial):
    sigma = trial.suggest_float("sigma", 0.1, 7.0)  # 使用 Optuna 建议 sigma 值
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = NetGELU(sigma).to(device)
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
    
        # 在测试集上验证模型
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        
        return accuracy
        

In [15]:
# 运行优化
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)

# 获取最佳 sigma 值
best_sigma = study.best_params["sigma"]
print("Best sigma:", best_sigma)

In [16]:
# 实例化模型、定义损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NetGELU(best_sigma).to(device)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 50


for epoch in range(num_epochs):
    model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')